In [1]:
from hope.dataset.dataset import CMPFacadeDataset
from hope.models.hope import HopeModel

/home/penguin/miniconda3/envs/research/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
model = HopeModel(512)

In [3]:
dataset = CMPFacadeDataset('/home/penguin/SSIW/data/base')

In [4]:
for e in dataset:
    image, mask = e
    break

/home/penguin/SSIW/data/base/cmp_b0346.jpg


In [5]:
logit, mask = model(image, mask)

torch.Size([205, 512])


In [6]:
logit.shape, mask.shape


(torch.Size([1, 205, 736, 960]), torch.Size([205, 736, 960]))

In [7]:
import torch

In [8]:
criteria = torch.nn.CrossEntropyLoss()

In [12]:
logit = logit.to('cpu')
mask = mask.to('cpu')

In [13]:
criteria(logit, mask.unsqueeze(0))

tensor(7.2520)